In [3]:
%load_ext autoreload
%autoreload 2

import numpy as np
from pathlib import Path
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from datetime import datetime
from scipy.cluster.hierarchy import linkage, fcluster, leaves_list
from scipy.spatial.distance import pdist
from sklearn.cluster import KMeans
import seaborn as sns
import src.stimuli_timeline as st
import src.plotting as plott
import src.data_loading as exio
import src.analysis_tools as at
from src.significant_traces import compute_noise_model_romano_fast_modular, plot_dff_and_raster
plt.style.use('default')
%matplotlib qt

In [4]:
# ==== CONFIG ================================================================
fish_id = "L433_f02"
experiment_name = "Exp_1_flickering"
main_path = Path(r"C:\Users\suribear\OneDrive - Université de Lausanne\Lab\Data\2p")
stimuli_main_path = Path(r"\\nasdcsr.unil.ch\RECHERCHE\FAC\FBM\CIG\jlarsch\default\D2c\Alejandro\2p")
fps_2p = 2.0
selected_blocks = [f"B{n}" for n in range(1, 3)]

# ==== LOAD EVERYTHING IN ONE GO ============================================
results = exio.load_2p_experiment(
    fish_id=fish_id,
    experiment_name=experiment_name,
    main_path=main_path,
    stimuli_main_path=stimuli_main_path,
    fps_2p=fps_2p,
    selected_blocks=selected_blocks,
)

dfof              = results["dfof"]
stimuli_durations = results["stimuli_durations"]
exp_log      = results["adjusted_log"]
stimuli_trace_60  = results["stimuli_trace_60"]
stimuli_table     = results["stimuli_table"]
stimuli_id_map    = results["stimuli_id_map"]
duration_2p_block_sec = results["duration_2p_block_sec"]
frames_per_block       = results["frames_per_block"]
paths                  = results["paths"]

print("dFoF shape:", dfof.shape)
print("Frames per block:", frames_per_block)
print("Paths:", paths["dfof_file"])

✅ Using preferred dFoF file: C:\Users\suribear\OneDrive - Université de Lausanne\Lab\Data\2p\L433_f02_Exp_1_flickering\03_analysis\functional\suite2P\merged_dFoF\L433_f02_dFoF_merged.npy
✅ Using block log: C:\Users\suribear\OneDrive - Université de Lausanne\Lab\Data\2p\L433_f02_Exp_1_flickering\01_raw\2p\metadata\2025-08-05-1506_L433_f02_block_log.csv
dFoF shape: (3340, 1984)
Frames per block: 1670
Paths: C:\Users\suribear\OneDrive - Université de Lausanne\Lab\Data\2p\L433_f02_Exp_1_flickering\03_analysis\functional\suite2P\merged_dFoF\L433_f02_dFoF_merged.npy


In [5]:
# select colors and linestyles for stimuli
# --- Set A (default) ----------------------------------------------------------
colors_A = {
    'LLB': sns.color_palette('Blues', 8)[7],
    'RLB': sns.color_palette('Greens', as_cmap=True)(0.6),
    'FLB': sns.color_palette('Blues', 8)[7],
    'FRB': sns.color_palette('Greens', as_cmap=True)(0.6),
    'FL1': sns.color_palette('PuRd', as_cmap=True)(0.9),
    'FL2': sns.color_palette('PuRd', as_cmap=True)(0.6),
    'FL3': sns.color_palette('dark:#404040', 10)[7],
    'FR1': sns.color_palette('PuRd', as_cmap=True)(0.9),
    'FR2': sns.color_palette('PuRd', as_cmap=True)(0.6),
    'FR3': sns.color_palette('dark:#404040', 10)[7],
}
linestyles_A = {
    'FLB': '--',
    'FRB': '--',
    'FL1': '--',
    'FL2': '--',
    'FL3': '--',
}

# --- Set B (rocking experiment) ----------------------------------------------
colors_B = {
    'LB':   sns.color_palette('Blues', 8)[7],
    'RR1':  sns.color_palette('Greens', as_cmap=True)(0.6),
    'RB':   sns.color_palette('Blues', 8)[7],
    'RL1':  sns.color_palette('Greens', as_cmap=True)(0.6),
    'RR2':  sns.color_palette('PuRd', as_cmap=True)(0.9),
    'RR3':  sns.color_palette('PuRd', as_cmap=True)(0.6),
    'RLR1': sns.color_palette('dark:#404040', 10)[7],
    'RL2':  sns.color_palette('PuRd', as_cmap=True)(0.9),   # fixed case from 'Rl2' → 'RL2'
    'RL3':  sns.color_palette('PuRd', as_cmap=True)(0.6),
    'RLR2': sns.color_palette('dark:#404040', 10)[7],
}
linestyles_B = {
    'LB': '--',
    'RL1': '--',
    'RL2': '--',
    'RL3': '--',
    'RLR2': '--',
}

# --- Select set based on experiment name --------------------------------------
if experiment_name == 'Exp_2_rocking_1':
    stimuli_colors = colors_B
    stimuli_linestyles = linestyles_B
else:
    stimuli_colors = colors_A
    stimuli_linestyles = linestyles_A

print("Using colors for:", experiment_name)
print(stimuli_linestyles)

Using colors for: Exp_1_flickering
{'FLB': '--', 'FRB': '--', 'FL1': '--', 'FL2': '--', 'FL3': '--'}





Plot

In [6]:
#plot stimuli as a binary trace
fps_stim=60
fig, ax = plt.subplots(figsize=(15, 2))
plott.add_stimuli_markers(ax, exp_log, stimuli_durations, stimuli_colors,stimuli_linestyles=stimuli_linestyles)
plt.title('Movement Periods')
plt.xlabel('Time (sec)')
plt.ylabel('Movement (Binary)')
plt.grid(True)
plt.ylim(0, 1.1)
plt.xlim(0, len(stimuli_trace_60) / fps_stim)
plt.tight_layout()
plt.show()


In [6]:
# # assume fps_stim is defined (e.g., 60) and ax is your matplotlib axes
# for stim_name, timing in stimuli_durations.items():
#     start_f = timing.get('start_frame', timing.get('motion_start_frame'))
#     end_f   = timing.get('end_frame',   timing.get('motion_end_frame'))
#     if start_f is None or end_f is None:
#         print(f"Skipping {stim_name}: no start/end frame keys")
#         continue
#
#     start = start_f / fps_stim
#     end   = end_f   / fps_stim
#     ax.axvspan(start, end, alpha=0.3)  # avoid hardcoding a color unless you want to


### Delta F - fluorescent traces

### Plots

##### All Planes raster plots with line for stimulus onset

In [23]:
# simple plot of delfta F over F with stimuli markers

fig, ax = plt.subplots(figsize=(12, 4))

# call your function — use vmax (or vmin/vmax) depending on your signature
im = plott.raster_with_stimuli(
    ax=ax,
    deltaF_F=dfof,
    fps=fps_2p,
    fish_id=fish_id,
    max=0.15,        # if your function still uses min/max, use min=0, max=0.4 instead
)

# add stimuli markers & legend on the SAME axes
legend_handles = plott.add_stimuli_markers(ax, exp_log, stimuli_durations, stimuli_colors,stimuli_linestyles=stimuli_linestyles)
ax.legend(handles=legend_handles, title="Stimuli",
          bbox_to_anchor=(1.2, 1), loc='upper left', borderaxespad=0, frameon=False)

ax.set_xlabel("Time (s)")

# add colorbar to the SAME figure
fig.colorbar(im, ax=ax, label="ΔF/F")

fig.tight_layout()
plt.show()


In [6]:
# --- choose stimulus-aligned windows ---
window_pre = 5  # seconds before stimulus
window_post = 17  # seconds after stimulus

# define stimuli order based on experiment
stimuli_ordered_A = ['LLB', 'FLB', 'RLB' , 'FRB','FL1', 'FL2', 'FL3', 'FR1', 'FR2', 'FR3']  # Order of
stimuli_ordered_B = ['LB','RB','RR1','RR2','RR3','RL1','RL2','RL3','RLR1','RLR2' ]

if experiment_name == 'Exp_2_rocking_1':
    stimuli_ordered = stimuli_ordered_B
else:
    stimuli_ordered = stimuli_ordered_A


In [10]:
# --- Plot sorted chunks for a single mode ---
# Sort by kmeans, pca, hier, covariance, max_intensity, none(unsorted), just modified sort mode
# for average_across_repeats = True or False

fig, ax, order = plott.plot_sorted_chunks_single_mode(
    dfof=dfof,
    exp_log=exp_log,
    stimuli_durations=stimuli_durations,
    stimuli_colors=stimuli_colors,
    stimuli_linestyles=stimuli_linestyles,
    stimuli_ordered=stimuli_ordered,
    fps_2p=fps_2p,
    window_pre=window_pre,
    window_post=window_post,
    sort_mode="kmeans",        # or 'pca', 'hier', 'corravg', max_intensity, kmeans, none(unsorted) ...
    n_clusters=5,
    random_state=42,
    average_across_repeats=True, # 👈 False to keep all repeats True to average across repeats
    fish_id=fish_id,
    neuron_order=None,       # 👈 custom order (optional)
    sort_label=None,         # 👈 label to show in the title
)
plt.show()


C:\Users\suribear\AppData\Local\anaconda3\envs\neuro-group\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=8.
  warnings.warn(


##### Grouped chuncks ordered per stimulus type

In [11]:
# Save sorted rasters for all modes
#  average_across_repeats = False # keep all repeats = True # average across repeats

plott.save_sorted_rasters_for_all_modes(
    dfof=dfof,
    exp_log=exp_log,
    stimuli_durations=stimuli_durations,
    stimuli_colors=stimuli_colors,
    stimuli_linestyles=stimuli_linestyles,
    stimuli_ordered=stimuli_ordered,
    fps_2p=fps_2p,
    window_pre=window_pre,
    window_post=window_post,
    plots_path=paths["plots_path"],
    prefix=paths["prefix"],
    n_clusters=3,
    random_state=42,
    average_across_repeats=False, # 👈 False to keep all repeats True to average across repeats
    folder_name='merge_dFoF' # 👈 additional folder name to identify the analysis
)


C:\Users\suribear\AppData\Local\anaconda3\envs\neuro-group\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=8.
  warnings.warn(


✅ Saved:
- L433_f02_grouped_dfof_sorted_by_unsorted.png
- L433_f02_grouped_dfof_sorted_by_max_intensity.png
- L433_f02_grouped_dfof_sorted_by_pca.png
- L433_f02_grouped_dfof_sorted_by_kmeans.png
- L433_f02_grouped_dfof_sorted_by_hier.png
- L433_f02_grouped_dfof_sorted_by_corravg.png


In [7]:
## Generate trial-aligned traces regarding stimuli
# Build trial-aligned traces, i.e., snippets of dF/F around each stimulus onset
# you define the pre- and post- time windows (in seconds)

res = at.build_trial_aligned_traces(
    dfof=dfof,
    stimuli_trace_60=stimuli_trace_60,
    fps_2p=fps_2p,
    t_pre_s=5.0,
    t_post_s=29.0,
    stimuli_id_map=stimuli_id_map,  # or None
    verbose=True
)

trial_aligned_traces = res["trial_aligned_traces"]
stimuli_ids          = res["stimuli_ids"]
stimuli_names        = res["stimuli_names"]
win_length           = res["win_length"]
stimuli_trace        = res["stimuli_trace"]
onsets_by_id         = res["onsets_by_id"]


Number of neurons: 1984
Number of time points: 3340
2p duration (s): 1670.0
60Hz duration from data (s): 1665.6333333333334
Output stim trace shape: (3340,)
Stimuli IDs: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Stimuli names: ['FL1', 'FL2', 'FL3', 'FLB', 'FR1', 'FR2', 'FR3', 'FRB', 'LLB', 'RLB']
stim 1: 4 onsets | kept 4 trials | dropped 0
stim 2: 4 onsets | kept 4 trials | dropped 0
stim 3: 4 onsets | kept 4 trials | dropped 0
stim 4: 4 onsets | kept 4 trials | dropped 0
stim 5: 4 onsets | kept 4 trials | dropped 0
stim 6: 4 onsets | kept 4 trials | dropped 0
stim 7: 4 onsets | kept 4 trials | dropped 0
stim 8: 4 onsets | kept 4 trials | dropped 0
stim 9: 4 onsets | kept 4 trials | dropped 0
stim 10: 4 onsets | kept 4 trials | dropped 0


In [26]:
# Filter neurons by trial reliability
# This function computes the trial reliability for each neuron and filters out
# those that do not meet a certain reliability threshold.
# It also generates plots to visualize the reliability distribution and
# the traces of kept vs. discarded neurons.
results = at.filter_neurons_by_trial_reliability(
    dfof=dfof,                        # shape (T, n_neurons)
    trial_aligned_traces=trial_aligned_traces,
    stimuli_ids=stimuli_ids,
    fps_2p=fps_2p,
    plots_path=paths["plots_path"],
    prefix=paths["prefix"],
    folder_name="filtered_neurons_by_stimuli",
    make_plots=True,      # switch to False if you don't want any plots
    save_indices=True,    # switch to False if you don't want to save the .npy
)

kept_neuron_indices = results["kept_neuron_indices"]
print(f"Kept {len(kept_neuron_indices)} neurons after reliability filtering.")

Saved kept neuron indices to: C:\Users\suribear\OneDrive - Université de Lausanne\Lab\Data\2p\L433_f02_Exp_1_flickering\03_analysis\functional\plots\filtered_neurons_by_stimuli\L433_f02_kept_neuron_indices.npy
Kept 641 neurons after reliability filtering.


In [10]:
## Save sorted rasters for all modes - filtered neurons only
# Save sorted rasters for all modes
#  average_across_repeats = False # keep all repeats = True # average across repeats

dfof_kept=dfof[:, kept_neuron_indices]
plott.save_sorted_rasters_for_all_modes(
    dfof=dfof_kept,
    exp_log=exp_log,
    stimuli_durations=stimuli_durations,
    stimuli_colors=stimuli_colors,
    stimuli_linestyles=stimuli_linestyles,
    stimuli_ordered=stimuli_ordered,
    fps_2p=fps_2p,
    window_pre=window_pre,
    window_post=window_post,
    plots_path=paths["plots_path"],
    prefix=paths["prefix"],
    n_clusters=3,
    random_state=42,
    average_across_repeats=False, # 👈 False to keep all repeats True to average across repeats
    folder_name='filtered_neurons_by_stimuli' # 👈 additional folder name to identify the analysis
)




C:\Users\suribear\AppData\Local\anaconda3\envs\neuro-group\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(


✅ Saved:
- L433_f02_grouped_dfof_sorted_by_unsorted.png
- L433_f02_grouped_dfof_sorted_by_max_intensity.png
- L433_f02_grouped_dfof_sorted_by_pca.png
- L433_f02_grouped_dfof_sorted_by_kmeans.png
- L433_f02_grouped_dfof_sorted_by_hier.png
- L433_f02_grouped_dfof_sorted_by_corravg.png


In [12]:
# Plot significant traces only - filtered neurons only
# Compute significant traces using the modified Romano et al. method

dfof_kept=dfof[:, kept_neuron_indices] # neurons responsive to stimuli only
FPS_DEFAULT = 2.0
TAUDECAY_DEFAULT = 6.0

ST_N_BINS       = 2000
ST_K_NEIGHBORS  = 100
ST_CONF_CUTOFF  = 90
ST_PLOT_ODDS    = False
ST_VMAX_DFF     = 0.15

(mapOfOdds,
         deltaF_center,
         density_data,
         density_noise,
         xev, yev,
         raster,
         mapOfOddsJoint) = compute_noise_model_romano_fast_modular(
             dfof_kept,
             n_bins=ST_N_BINS,
             k_neighbors=ST_K_NEIGHBORS,
             confCutOff=ST_CONF_CUTOFF,
             plot_odds=ST_PLOT_ODDS,
             fps=FPS_DEFAULT,
             tauDecay=TAUDECAY_DEFAULT,
        )

prefix=paths["prefix"]
folder_raster = paths["plots_path"] / 'significant_traces'
folder_raster.mkdir(parents=True, exist_ok=True)

out_raster = folder_raster / f"{prefix}_significant_traces.npz"
np.savez(
    out_raster,
    raster=raster,
    deltaF_center=deltaF_center,
)

print(f"Saved significant traces to {out_raster}")


[Noise Fit] Used fallback on 0 out of 641 ROIs (0.0%)


In [13]:
## plot example dF/F and raster to check if significant traces look ok.

sort_idx = plot_dff_and_raster(deltaF_center[:,1:100], raster[:,1:100], fps=2.0, vmax_dff=0.3)

# Sanity check before pdist
n_neurons =raster.shape[1] #  frames x nueroso
nan_mask = ~np.isfinite(raster).all(axis=0)
zero_var_mask = np.nanstd(raster, axis=0) == 0

print("Neurons with any NaN:", np.where(nan_mask)[0])
print("Neurons with zero variance:", np.where(zero_var_mask)[0])
print("Total bad neurons:", np.sum(nan_mask | zero_var_mask), "out of", n_neurons)

Neurons with any NaN: []
Neurons with zero variance: [74]
Total bad neurons: 1 out of 641


In [22]:

# here for plotting significant traces only
# --- Plot sorted chunks for a single mode ---
# Sort by kmeans, pca, hier, covariance, max_intensity, none(unsorted), just modified sort mode
# for average_across_repeats = True or False

raster_clean = raster[:,~zero_var_mask ] # id there is not variability of NaN correlation does not work

fig, ax, order = plott.plot_sorted_chunks_single_mode(
    dfof=raster_clean,
    exp_log=exp_log,
    stimuli_durations=stimuli_durations,
    stimuli_colors=stimuli_colors,
    stimuli_linestyles=stimuli_linestyles,
    stimuli_ordered=stimuli_ordered,
    fps_2p=fps_2p,
    window_pre=window_pre,
    window_post=window_post,
    sort_mode="corravg",        # or 'pca', 'hier', 'corravg', max_intensity, kmeans, none(unsorted) ...
    n_clusters=3,
    random_state=42,
    average_across_repeats=False, # 👈 False to keep all repeats True to average across repeats
    fish_id=fish_id,
    neuron_order=None,       # 👈 custom order (optional)
    sort_label=None,         # 👈 label to show in the title
)
plt.show()


In [34]:
raster_clean = raster[:,~zero_var_mask ] # id there is not variability of NaN correlation does not work
plott.save_sorted_rasters_for_all_modes(
    dfof=raster_clean,
    exp_log=exp_log,
    stimuli_durations=stimuli_durations,
    stimuli_colors=stimuli_colors,
    stimuli_linestyles=stimuli_linestyles,
    stimuli_ordered=stimuli_ordered,
    fps_2p=fps_2p,
    window_pre=window_pre,
    window_post=window_post,
    plots_path=paths["plots_path"],
    prefix=paths["prefix"],
    n_clusters=3,
    random_state=42,
    average_across_repeats=False, # 👈 False to keep all repeats True to average across repeats
    folder_name='significant_traces' # 👈 additional folder name to identify the analysis
)


C:\Users\suribear\AppData\Local\anaconda3\envs\neuro-group\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(


✅ Saved:
- L433_f02_grouped_dfof_sorted_by_unsorted.png
- L433_f02_grouped_dfof_sorted_by_max_intensity.png
- L433_f02_grouped_dfof_sorted_by_pca.png
- L433_f02_grouped_dfof_sorted_by_kmeans.png
- L433_f02_grouped_dfof_sorted_by_hier.png
- L433_f02_grouped_dfof_sorted_by_corravg.png


In [307]:
39830830# # Define neuron groups (in sorted neuron indices)
neuron_groups = {
    'C1 Left Bout': (140, 185),
    'C2 Right Bout': (85, 130),
    'C3 Continous': (65, 85)
}

# Colors for each group
group_colors = {
    'C1 Left Bout': 'green',
    'C2 Right Bout': 'firebrick',
    'C3 Continous': 'steelblue' #
}

# Extract stimulus-aligned chunks (already averages across repeats)
chunked_data, trial_starts, move_starts, move_colors, stim_labels = st.extract_stimulus_chunks(
    deltaF_F=fish_data['plane3']['deltaF_F'],
    exp_log=exp_log,
    stimuli_durations=stimuli_durations,
    stimuli_colors=stimuli_colors,
    fps=fps_2p,
    stimuli_ordered=stimuli_ordered,
    window_pre=12,
    window_post=2,
    average_across_repeats=True
)

dist = pdist(chunked_data, metric='correlation')
Z = linkage(dist, method='average')
sorted_indices = leaves_list(Z)

selected_chuncks = 4*70

# Apply sorting
#sorted_data = chunked_data[sorted_indices, :selected_chuncks]
sorted_data = chunked_data[sorted_indices, :]

fig, ax = plt.subplots(figsize=(9, 6))
time_axis = np.arange(sorted_data.shape[1]) / fps_2p

for cluster_name, (start_idx, end_idx) in neuron_groups.items():
    cluster_data = sorted_data[start_idx:end_idx, :]  # select neurons in sorted space
    mean_trace = np.nanmean(cluster_data, axis=0)
    ax.plot(time_axis, mean_trace, color=group_colors[cluster_name], linewidth=2, label=cluster_name)


# Add vertical lines for movement onsets
for pos, color in zip(move_starts[:4], move_colors[:4]):
    ax.axvline(pos / fps_2p, color=color, alpha=0.9, linewidth=2.5, linestyle='--')

# Create legend
legend_handles = []
for stim_name, color in stimuli_colors.items():
    line, = ax.plot([], [], color=color, label=stim_name, linewidth=4)
    legend_handles.append(line)

ax.legend(handles=legend_handles, title="Movement onset \n across stimuli",
          bbox_to_anchor=(1.2, 1), loc='upper left', borderaxespad=0, frameon=False)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.set_ylabel("Average ΔF/F")
ax.set_title(f"{fish_id} Plane 3 Average ΔF/F")
plt.subplots_adjust(right=0.85)
# fig.savefig(fig_paths / f'delta_average_plane3_allconditions.pdf', dpi=900)
# fig.savefig(fig_paths / f'delta_average_plane3_allconditions.png', dpi=1000)
plt.show()

NameError: name 'fish_data' is not defined

#### Trace ROIs back to anatomy

In [ ]:
stat = np.load(segmented_path / "plane3" / "stat.npy", allow_pickle=True) #load segmented ROIs
ops = np.load(segmented_path / "plane3" / "ops.npy", allow_pickle=True).item() #
mean_img = ops['meanImg']
roi_indices = fish_data['plane3']['ROIs_selected']

In [ ]:
chunked_data, trial_starts, move_starts, move_colors, stim_labels = st.extract_stimulus_chunks(
        deltaF_F=fish_data['plane3']['deltaF_F'],
        exp_log=exp_log,
        stimuli_durations=stimuli_durations,
        stimuli_colors=stimuli_colors,
        fps=fps_2p,
        stimuli_ordered=stimuli_ordered,
        window_pre=5,
        window_post=2,
        average_across_repeats=True)

dist = pdist(chunked_data, metric='correlation')
Z = linkage(dist, method='average')
sorted_indices = leaves_list(Z)

# Plot with sorted neurons
fig, ax = plt.subplots(figsize=(9, 6))
plott.raster_with_stimuli(
    ax=ax,
    deltaF_F=chunked_data.T,
    fps=fps_2p,
    plane_name='Plane 3',
    fish_id=fish_id,
    neuron_order=sorted_indices,
    title_suffix='Average across trials \n sorted by Pearson correlation'
)

for pos, color in zip(move_starts, move_colors):
    ax.axvline(pos / fps_2p, color=color, alpha=0.9, linewidth=1.5)

legend_handles = []
for stim_name, color in stimuli_colors.items():
    line, = ax.plot([], [], color=color, label=stim_name, linewidth=4)
    legend_handles.append(line)
ax.legend(handles=legend_handles, title="Movement onset \n across stimuli", bbox_to_anchor=(1.2, 1), loc='upper left', borderaxespad=0, frameon=False)
plt.subplots_adjust(right=0.85)
fig.savefig(fig_paths / f'raster_average_pearson_{plane_name}.pdf', dpi=900)
fig.savefig(fig_paths / f'raster_average_pearson_{plane_name}.png', dpi=1000)
plt.show()

In [ ]:
neuron_groups = {
    'C1 Left Bout': (140, 185),
    'C2 Right Bout': (85, 130),
    'C3 Continous': (65, 85)
}

# Colors for each group
group_colors = {
    'C1 Left Bout': 'green',
    'C2 Right Bout': 'firebrick',
    'C3 Continous': 'steelblue' #
}

In [ ]:
selected_neurons_sorted = np.arange(90, 135) #140, 185 left B,  130-190 right B
original_indices = roi_indices[sorted_indices[selected_neurons_sorted]]

fig, ax = plt.subplots(figsize=(6, 6))

p1, p99 = np.percentile(mean_img, (1, 99))
mean_img_clipped = np.clip(mean_img, p1, p99)
mean_img_norm = (mean_img_clipped - p1) / (p99 - p1)
plt.imshow(mean_img_norm, cmap='gray')

for i in original_indices:
    xcen = stat[i]['xpix'].mean()
    ycen = stat[i]['ypix'].mean()
    plt.plot(xcen, ycen, 'go', markersize=5)

plt.title(f"Plane 3 - ROIs {min(selected_neurons_sorted)}–{max(selected_neurons_sorted)} \n sorted by Pearson correlation")
ax.axis('off')
plt.tight_layout()
# fig.savefig(fig_paths / f'{plane_name}_ROIs_{min(selected_neurons_sorted)}–{max(selected_neurons_sorted)}_anatomy.pdf', dpi=900, bbox_inches='tight')
# fig.savefig(fig_paths / f'{plane_name}_ROIs_{min(selected_neurons_sorted)}–{max(selected_neurons_sorted)}_anatomy.png', dpi=1000, bbox_inches='tight')
plt.show()

In [ ]:
# fig, ax = plt.subplots(figsize=(6, 6))
#
# # Normalize mean image for background
# p1, p99 = np.percentile(mean_img, (1, 99))
# mean_img_clipped = np.clip(mean_img, p1, p99)
# mean_img_norm = (mean_img_clipped - p1) / (p99 - p1)
# plt.imshow(mean_img_norm, cmap='gray')
#
# # Loop over neuron groups
# for cluster_name, (start_idx, end_idx) in neuron_groups.items():
#     sorted_neurons = np.arange(start_idx, end_idx)
#     original_indices = roi_indices[sorted_indices[sorted_neurons]]
#
#     for i in original_indices:
#         xcen = stat[i]['xpix'].mean()
#         ycen = stat[i]['ypix'].mean()
#         plt.plot(xcen, ycen, 'o', color=group_colors[cluster_name], markersize=5, label=cluster_name)
#
# plt.title(f"Plane 3 - Anatomical location of selected ROIs")
# ax.axis('off')
# plt.tight_layout()
# fig.savefig(fig_paths / f'plane3_ROIs_clusters_anatomy.pdf', dpi=900, bbox_inches='tight')
# fig.savefig(fig_paths / f'plane3_ROIs_clusters_anatomy.png', dpi=1000, bbox_inches='tight')
# plt.show()

Average